In [ ]:

import os
import pandas as pd
from google.colab import files
from pydub import AudioSegment
from speechbrain.pretrained import SpeakerRecognition
import ipywidgets as widgets
from IPython.display import display, clear_output

# 🎯 Cargar modelo
verification = SpeakerRecognition.from_hparams(
    source="speechbrain/spkrec-ecapa-voxceleb",
    savedir="pretrained_models/spkrec-ecapa-voxceleb"
)

# 🔼 Subir archivos
print("🔼 Sube varios archivos de audio (voz real + sospechosas):")
uploaded = files.upload()

# Convertir a WAV si es necesario
def convert_to_wav(input_file):
    sound = AudioSegment.from_file(input_file)
    output_file = os.path.splitext(input_file)[0] + "_converted.wav"
    sound.export(output_file, format="wav")
    return output_file

# Preparar todos los archivos
file_list = list(uploaded.keys())
prepared_files = []

for fname in file_list:
    ext = os.path.splitext(fname)[1].lower()
    prepared = convert_to_wav(fname) if ext != ".wav" else fname
    prepared_files.append(prepared)
    os.remove(fname)  # Eliminar original tras conversión

# 🎚️ Selector de voz base (widget)
selector = widgets.Dropdown(
    options=prepared_files,
    description='🎙️ Voz base:',
    layout=widgets.Layout(width='50%')
)

button = widgets.Button(description="🔍 Comparar con otras voces")

# 🔍 Clasificar nivel de confianza
def interpretar_confianza(score):
    if score < 0.50:
        return "🔴 Muy baja"
    elif score < 0.60:
        return "🟠 Baja"
    elif score <= 0.70:
        return "🟡 Media"
    elif score <= 0.80:
        return "🔵 Alta"
    else:
        return "🟣 Muy alta"

# ▶️ Lógica al pulsar el botón
def on_button_clicked(b):
    clear_output(wait=True)
    display(selector, button)

    archivo_base = selector.value
    print(f"\n🔍 Comparando voz base: {archivo_base}...\n")

    resultados = []

    for archivo in prepared_files:
        if archivo == archivo_base:
            continue
        score, prediction = verification.verify_files(archivo_base, archivo)
        score = score.item()
        confianza = interpretar_confianza(score)
        resultados.append({
            "Voz base": archivo_base,
            "Voz comparada": archivo,
            "Score": round(score, 4),
            "¿Misma persona?": "Sí" if prediction else "No",
            "Confianza": confianza
        })

    # Crear DataFrame y mostrar
    df = pd.DataFrame(resultados)
    display(df)

    # Guardar CSV
    csv_path = "resultados_comparacion.csv"
    df.to_csv(csv_path, index=False)
    print(f"\n💾 Resultados guardados en: {csv_path}")
    files.download(csv_path)

    # Limpiar temporales
    for f in prepared_files:
        if f.endswith("_converted.wav"):
            os.remove(f)

# Enlazar botón al evento
button.on_click(on_button_clicked)

# Mostrar interfaz
display(selector, button)
